## Verifying treecorr patch stochasiticity's effect on jackknife

- Defining patches in catalogue affects resulting values due to different numerical realization
- Stochasticity in patches has significant bearing on resulting tangential shear values of similar order to 
    signal magnitude 
- Tried adding patches into catalogues to see if it boosted SNR

In [1]:
import numpy as np
import treecorr
import des_functions as des
from more_itertools import locate
import time
import astropy.io.fits as fits 
import matplotlib.pyplot as plt

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['cmr10']
plt.rcParams['mathtext.fontset'] ='cm'
data_dir = '/home/b7009348/WGL_project/DES-data/'

In [4]:
#des.create_patches(lens_file=lens_file, npatches=npatches)

Patches saved, runtime=1.197099.


In [2]:
lens_file = 'DES_Y1A1_Lenses_z=0.3-0.45.fits'
rand_file = 'DES_Y1A1_3x2pt_redMaGiC_RANDOMS.fits'
im3_file = 'y1_im3_shapes_matched.fits'
mcal_file = 'y1_mcal_shapes_matched.fits'
z_file = 'mcal_BPZ_(binning).fits'

In [3]:
npatches= 20
sep_bins = 20
theta_min = 2.5
theta_max = 250.0
slops = [0]

In [ ]:
start=time.time()

with fits.open(data_dir+lens_file) as hdu:
    data = hdu[1].data
    ra_l = data['RA']
    dec_l = data['DEC']
    w_l = data['weight']
del data

cat_l = treecorr.Catalog(ra=ra_l, dec=dec_l, ra_units='deg', dec_units='deg', w=w_l, patch_centers=data_dir+'jackknife_patch_centers')

del ra_l, dec_l, w_l

with fits.open(data_dir+rand_file) as hdu:
    data = hdu[1].data
    indexes = list(locate(data['z'], lambda x: 0.3 < x < 0.45))
    ra_r = data['RA'][indexes]
    dec_r = data['DEC'][indexes]
    rand_z = data['z'][indexes]
del data

cat_r = treecorr.Catalog(ra=ra_r, dec=dec_r, ra_units='deg', dec_units='deg', r=rand_z, patch_centers=data_dir+'jackknife_patch_centers')

del indexes, ra_r, dec_r, rand_z

with fits.open(data_dir+mcal_file) as hdu:
    data = hdu[1].data
    ra_s = data['ra']
    dec_s = data['dec']
    e1 = data['e1']
    e2 = data['e2']
    R11 = data['R11']
del data

cat_mcal = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', r=R11, g1=e1, g2=e2, patch_centers=data_dir+'jackknife_patch_centers')

del ra_s, dec_s, e1, e2, R11

with fits.open(data_dir+im3_file) as hdu:
    data = hdu[1].data
    im3_ID = data['coadd_objects_id']
    ra_s = data['ra']
    dec_s = data['dec']
    e1 = data['e1'] - data['c1']
    e2 = data['e2'] - data['c2']
    sens = data['m'] + 1.0
    w = data['weight']
del data

with fits.open(data_dir+z_file) as hdu:
    data = hdu[1].data
    zmc = data['z_mc']
    zmc_ID = data['coadd_objects_id']
del data

matches, zmc_indices, shape_indices = np.intersect1d(zmc_ID, im3_ID, return_indices=True)

zmc = zmc[zmc_indices]

del zmc_ID, matches, zmc_indices, shape_indices, im3_ID

cat_im3 = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', r=zmc, g1=e1, g2=e2, w=w, 
                           patch_centers=data_dir+'jackknife_patch_centers')
cat_k = treecorr.Catalog(ra=ra_s, dec=dec_s, ra_units='deg', dec_units='deg', k=sens, w=w, patch_centers=data_dir+'jackknife_patch_centers')

del ra_s, dec_s, e1, e2, sens, zmc, w

end = time.time()
diff=end-start

print('Catalogues created, runtime=%f.'%diff)

IA_final = np.zeros([len(slops),sep_bins])
IA_sig = np.zeros_like(IA_final)

log_shift = np.logspace(np.log10(0.025), np.log10(2.5), 20)
    
IA_jk, IA_sig = des.IA_jackknife(cat_l=cat_l, cat_r=cat_r, cat_mcal=cat_mcal, cat_im3=cat_im3, cat_k=cat_k, sep_bins=sep_bins, npatches=npatches, fbins=810,
                                    theta_min=theta_min, theta_max=theta_max, bin_slop=slops[0])

IA_final, theta = des.IA_full(cat_l=cat_l, cat_r=cat_r, cat_mcal=cat_mcal, cat_im3=cat_im3, cat_k=cat_k, sep_bins=sep_bins, fbins=810, 
                                  theta_min=theta_min, theta_max=theta_max, bin_slop=slops[0])                           


plt.figure(figsize=[6,5])
plt.errorbar(theta, IA_final, yerr=IA_sig, lw=0, marker='o', color='orange', markersize=5, elinewidth=1, capsize=3)    
plt.yscale('log')
plt.xscale('log')
plt.ylim([1e-6, 1e-2])
plt.xlim([2.5,250])
plt.xlabel(r'$\theta$ (arcmin)')
plt.ylabel(r'$\bar{\gamma}_{IA}(\theta)$')
plt.title(r'$0.30<z_{l}<0.45$')    
plt.legend(['Bin slop = 0'])
plt.savefig('IA_patches_BinSlop=0_patches.png', dpi=300)
    
print(IA_final, IA_sig) 

del cat_l, cat_r, cat_im3, cat_mcal, cat_k

Catalogues created, runtime=34.486082.
Patch 0 located and sliced, calculating correlations...
The following patch numbers have no objects: {0}
This may be a problem depending on your use case.
The following patch numbers have no objects: {0}
This may be a problem depending on your use case.
The following patch numbers have no objects: {0}
This may be a problem depending on your use case.
The following patch numbers have no objects: {0}
This may be a problem depending on your use case.
IA signal estimated, runtime=1746.303084.
Patch 1 located and sliced, calculating correlations...
The following patch numbers have no objects: {1}
This may be a problem depending on your use case.
The following patch numbers have no objects: {1}
This may be a problem depending on your use case.
The following patch numbers have no objects: {1}
This may be a problem depending on your use case.
The following patch numbers have no objects: {1}
This may be a problem depending on your use case.
IA signal estim

In [5]:
jackknife_values = np.load(data_dir+'ia_jackknife_values-bin_slop=0_(patches).npz')

In [6]:
cov = jackknife_values['IA_cov']

In [7]:
print(cov)

[[ 3.04306410e-07  1.38887996e-07  7.54709636e-08  4.25352933e-08
   1.97701905e-08  9.23477657e-08  3.40063109e-08  1.25461945e-08
   1.99949133e-08 -8.47713723e-09  1.08938654e-10  4.94679463e-09
   2.74579735e-08  5.93440827e-09  1.17129974e-08  1.20188328e-08
   1.64784285e-09  5.48865857e-09  7.76374991e-09  7.32301883e-09]
 [ 1.38887996e-07  2.59213522e-07  1.60305253e-07  8.82778507e-08
   9.39250162e-08  1.24833707e-07  5.93240095e-08  1.66384108e-08
   5.29588147e-08  2.45172757e-08  2.10201388e-08  1.54031519e-08
   4.23101884e-08  2.16177234e-08  2.64468442e-08  3.24722963e-08
   2.21793877e-08  2.08754690e-08  1.36513742e-08  1.25525380e-08]
 [ 7.54709636e-08  1.60305253e-07  3.26460863e-07  7.76499183e-08
   1.00155653e-07  9.26002216e-08  7.78123439e-08  2.19348428e-08
   5.84344044e-08  5.09774264e-08  1.69618028e-08  2.90107150e-08
   4.24027661e-08  1.06906048e-08  2.22449448e-08  3.66304490e-08
   3.03278029e-08  2.70747662e-08  1.25644103e-08  1.27922059e-09]
 [ 4.25